In [1]:
import cProfile
import os

In [2]:
import brutus
from brutus import fitting, filters
from brutus.utils import inv_magnitude, load_models, load_offsets

import h5py
import numpy as np

In [3]:
filt = filters.ps[:-2] + filters.tmass

# load in models
(models_mist, labels_mist,
 lmask_mist) = load_models('../data/DATAFILES/grid_mist_v8.h5', 
                                  filters=filt, include_ms=True, 
                                  include_postms=True, include_binaries=True)

# load in offsets
zp_mist = load_offsets('../data/DATAFILES/offsets_mist_v8.txt', 
                                 filters=filt)

# load in fitter
BF_mist = fitting.BruteForce(models_mist, labels_mist, lmask_mist)

Reading filter 2MASS_Ks           
PS_g (0.0%)
PS_r (-3.0%)
PS_i (-3.0%)
PS_z (-2.0%)
PS_y (-4.0%)
2MASS_J (-2.0%)
2MASS_H (1.0%)
2MASS_Ks (-2.0%)


In [4]:
# load in data
filename = 'Orion_l204.7_b-19.2'
f = h5py.File(filename+'.h5', mode='r')
fpix = f['photometry']['pixel 0-0']
mag, magerr = fpix['mag'], fpix['err']
mask = np.isfinite(magerr)  # create boolean band mask
phot, err = inv_magnitude(mag, magerr)  # convert to flux
objid = fpix['obj_id']
parallax, parallax_err = fpix['parallax'] * 1e3, fpix['parallax_error'] * 1e3  # convert to mas
psel = np.isclose(parallax_err, 0.) | np.isclose(parallax, 0.) | (parallax_err > 1e6)
parallax[psel], parallax_err[psel] = np.nan, np.nan
coords = np.c_[fpix['l'], fpix['b']]

/Users/abeane/scratch/brutus/brutus/utils.py:504: RuntimeWarning: overflow encountered in power
  phot = 10**(-0.4 * mag) * zeropoints


In [5]:

if os.path.exists('Orion_l204.7_b-19.2_mist.h5'):
    os.remove('Orion_l204.7_b-19.2_mist.h5')

cProfile.run("BF_mist.fit(phot[:10], err[:10], mask[:10], objid[:10],\
              filename+'_mist',\
              parallax=parallax[:10], parallax_err=parallax_err[:10],\
              data_coords=coords[:10],\
              av_gauss=(0,1e6),\
              phot_offsets=zp_mist,\
              running_io=True,\
              save_dar_draws=True,\
              verbose=True)", "profiles/profile.dat")

Fitting object 10/10 [chi2/n: 1.6/6] (mean time: 1.653 s/obj, est. time remaining:      0.000 s)    


In [ ]:
            # dustfile='../data/DATAFILES/bayestar2019_v1.h5',\

In [6]:
import pstats
from pstats import SortKey

In [7]:
p = pstats.Stats('profiles/profile.dat')
# p.strip_dirs().sort_stats(-1).print_stats()

In [8]:
p.sort_stats(SortKey.TIME).print_stats()

Sun May 17 11:11:02 2020    profiles/profile.dat

         456429 function calls (452306 primitive calls) in 21.938 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       59    7.281    0.123   11.050    0.187 /Users/abeane/scratch/brutus/brutus/fitting.py:270(_optimize_fit)
        1    5.692    5.692    5.692    5.692 {method 'argsort' of 'numpy.ndarray' objects}
       69    2.142    0.031    2.239    0.032 /Users/abeane/scratch/brutus/brutus/utils.py:623(get_seds)
     9826    1.867    0.000    1.867    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       10    1.133    0.113   12.697    1.270 /Users/abeane/scratch/brutus/brutus/fitting.py:30(loglike)
        4    0.896    0.224    0.896    0.224 /usr/local/lib/python3.7/site-packages/h5py/_hl/dataset.py:476(__getitem__)
    17720    0.312    0.000    0.315    0.000 {built-in method numpy.array}
      950    0.218    0.000    0.218    0.000 {function Quantity.__array

       90    0.000    0.000    0.001    0.000 /usr/local/lib/python3.7/site-packages/astropy/coordinates/baseframe.py:1798(__setattr__)
      114    0.000    0.000    0.012    0.000 <__array_function__ internals>:2(cross)
      190    0.000    0.000    0.000    0.000 /usr/local/lib/python3.7/site-packages/astropy/coordinates/transformations.py:424(lookup_name)
       10    0.000    0.000    0.038    0.004 /usr/local/lib/python3.7/site-packages/astropy/coordinates/representation.py:1930(from_cartesian)
      122    0.000    0.000    0.000    0.000 /usr/local/lib/python3.7/site-packages/numpy/lib/stride_tricks.py:262(<listcomp>)
       10    0.000    0.000    0.001    0.000 /usr/local/lib/python3.7/site-packages/astropy/coordinates/baseframe.py:1784(__init__)
      732    0.000    0.000    0.000    0.000 {method 'keys' of 'collections.OrderedDict' objects}
      151    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
      800    0.000    0.000    0.000    0.000 /usr/l

        3    0.000    0.000    0.000    0.000 /usr/local/lib/python3.7/site-packages/h5py/_hl/compat.py:103(filename_encode)
       40    0.000    0.000    0.000    0.000 /usr/local/lib/python3.7/site-packages/astropy/coordinates/representation.py:1092(to_cartesian)
       46    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
       12    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py:240(__enter__)
       12    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py:249(_release_save)
       12    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py:252(_acquire_restore)
        5    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/fnmatch.py:64(fnmatc

In [11]:
p.sort_stats(SortKey.CUMULATIVE).print_stats(30)

Sun May 17 01:02:49 2020    profiles/profile.dat

         458299 function calls (454056 primitive calls) in 23.702 seconds

   Ordered by: cumulative time
   List reduced from 719 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   23.702   23.702 {built-in method builtins.exec}
        1    0.000    0.000   23.702   23.702 <string>:1(<module>)
        1    0.036    0.036   23.702   23.702 /Users/abeane/scratch/brutus/brutus/fitting.py:606(fit)
       11    0.146    0.013   16.315    1.483 /Users/abeane/scratch/brutus/brutus/fitting.py:1054(_fit)
       20    0.000    0.000   15.823    0.791 /Users/abeane/scratch/brutus/brutus/utils.py:45(__call__)
       10    0.060    0.006   13.975    1.397 /Users/abeane/scratch/brutus/brutus/fitting.py:1260(_loglike_zip)
       10    1.399    0.140   13.915    1.391 /Users/abeane/scratch/brutus/brutus/fitting.py:31(loglike)
       59    7.772    0.132   11.711    0.19

In [40]:
p.sort_stats(SortKey.CUMULATIVE).print_stats('prior')

Fri May 15 14:55:17 2020    profiles/profile.dat

         6472409 function calls (6185314 primitive calls) in 34.316 seconds

   Ordered by: cumulative time
   List reduced from 5910 to 5 due to restriction <'prior'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       11    0.054    0.005    7.016    0.638 /Users/abeane/scratch/brutus/brutus/pdf.py:733(dust_lnprior)
       10    0.156    0.016    1.086    0.109 /Users/abeane/scratch/brutus/brutus/pdf.py:467(gal_lnprior)
       10    0.050    0.005    0.054    0.005 /Users/abeane/scratch/brutus/brutus/pdf.py:178(scale_parallax_lnprior)
        1    0.035    0.035    0.037    0.037 /Users/abeane/scratch/brutus/brutus/pdf.py:38(imf_lnprior)
       10    0.009    0.001    0.009    0.001 /Users/abeane/scratch/brutus/brutus/pdf.py:144(parallax_lnprior)




In [17]:
p_old = pstats.Stats('profiles/profile_670dd1361e.dat')
p_old.sort_stats(SortKey.TIME).print_stats(30)

Fri May 15 14:51:13 2020    profiles/profile_670dd1361e.dat

         6472409 function calls (6185314 primitive calls) in 34.316 seconds

   Ordered by: internal time
   List reduced from 5910 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       59    8.079    0.137   12.572    0.213 /Users/abeane/scratch/brutus/brutus/fitting.py:269(_optimize_fit)
        1    5.668    5.668    5.668    5.668 {method 'argsort' of 'numpy.ndarray' objects}
   209763    3.014    0.000    3.014    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       69    2.506    0.036    2.628    0.038 /Users/abeane/scratch/brutus/brutus/utils.py:609(get_seds)
       22    1.956    0.089    1.961    0.089 /usr/local/lib/python3.7/site-packages/h5py/_hl/dataset.py:476(__getitem__)
       10    1.765    0.177   15.382    1.538 /Users/abeane/scratch/brutus/brutus/fitting.py:31(loglike)
    49858    1.604    0.000    6.632    0.000 {method 'multivariate_normal' of 